In [1]:
# @title Helper functions for VLMap Creation

import os
import math

import open3d as o3d
import numpy as np
import cv2
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import clip

from scipy.spatial.transform import Rotation as R

from utils.clip_mapping_utils import load_semantic, load_obj2cls_dict, save_map, cvt_obj_id_2_cls_id, depth2pc_realsense, transform_pc, get_real_cam_mat, pos2grid_id, project_point, get_color
# from utils.clip_mapping_utils import load_pose

from lseg.modules.models.lseg_net import LSegEncNet
from lseg.additional_utils.models import resize_image, pad_image, crop_image


def load_depth(depth_filepath):
    with open(depth_filepath, 'rb') as f:
        depth = np.load(f)
    return depth / 1000
    # return depth

def load_pose(pose_filepath):
    with open(pose_filepath, 'rb') as f:
        full_pose = np.load(f)
        pos = np.array(full_pose[:3], dtype=float).reshape((3, 1))
        quat = full_pose[3:]
        # quat[[1,2]] = quat[[2,1]]
        r = R.from_quat(quat)
        rot = r.as_matrix()

        return pos, rot

# @title Helper functions for VLMap Creation

import os
import math

import numpy as np
import cv2
from tqdm import tqdm
import torch
import torchvision.transforms as transforms
import clip
import matplotlib.pyplot as plt

from utils.clip_mapping_utils import load_semantic, load_obj2cls_dict, save_map, cvt_obj_id_2_cls_id, depth2pc_realsense, transform_pc, get_real_cam_mat, pos2grid_id, project_point, get_color
# from utils.clip_mapping_utils import load_pose

from lseg.modules.models.lseg_net import LSegEncNet
from lseg.additional_utils.models import resize_image, pad_image, crop_image

import open3d as o3d

def create_lseg_map_batch(img_save_dir, camera_translation, ground_to_camera_height, cs=0.05, gs=1000, depth_sample_rate=100):
    mask_version = 1 # 0, 1

    # crop_size = 480 # 480
    # base_size = 520 # 520
    # lang = "door,chair,ground,ceiling,window,other"
    # labels = lang.split(",")

    # # loading models
    # device = "cuda:1" if torch.cuda.is_available() else "cpu"
    # print(device)
    # clip_version = "ViT-B/32"
    # clip_feat_dim = {'RN50': 1024, 'RN101': 512, 'RN50x4': 640, 'RN50x16': 768,
    #                 'RN50x64': 1024, 'ViT-B/32': 512, 'ViT-B/16': 512, 'ViT-L/14': 768}[clip_version]
    # print("Loading CLIP model...")
    # clip_model, preprocess = clip.load(clip_version, device = device)  # clip.available_models()
    # clip_model.to(device).eval()
    # # print("Clip model -", clip_model.is_cuda())
    # # x = torch.randn((5,5)).to(device=device)
    # # print("Tensor device - ", type(x))
    # # clip_model.eval()
    # lang_token = clip.tokenize(labels)
    # lang_token = lang_token.to(device)
    # with torch.no_grad():
    #     text_feats = clip_model.encode_text(lang_token)
    #     text_feats = text_feats / text_feats.norm(dim=-1, keepdim=True)
    # text_feats = text_feats.cpu().numpy()
    # model = LSegEncNet(lang, arch_option=0,
    #                     block_depth=0,
    #                     activation='lrelu',
    #                     crop_size=crop_size)
    # model_state_dict = model.state_dict()
    # global data_root
    # checkpoint_path = os.path.join(data_root, "lseg/checkpoints/demo_e200.ckpt")

    # print("Loading lseg checkpoint")
    # pretrained_state_dict = torch.load(checkpoint_path, map_location = device)
    # print("Lseg loaded")
    # pretrained_state_dict = {k.lstrip('net.'): v for k, v in pretrained_state_dict['state_dict'].items()}
    # model_state_dict.update(pretrained_state_dict)
    # model.load_state_dict(pretrained_state_dict)

    # model.eval()
    # model = model.cuda()

    # norm_mean= [0.5, 0.5, 0.5]
    # norm_std = [0.5, 0.5, 0.5]
    # padding = [0.0] * 3
    # transform = transforms.Compose(
    #     [
    #         transforms.ToTensor(),
    #         transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    #     ]
    # )

    print(f"loading scene {img_save_dir}")
    rgb_dir = os.path.join(img_save_dir, "rgb")
    depth_dir = os.path.join(img_save_dir, "depth")
    pose_dir = os.path.join(img_save_dir, "pose")
    # semantic_dir = os.path.join(img_save_dir, "semantic")
    # obj2cls_path = os.path.join(img_save_dir, "obj2cls_dict.txt")

    rgb_list = sorted(os.listdir(rgb_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    depth_list = sorted(os.listdir(depth_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    # pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
    #     x.split("_")[-1].split(".")[0]))
    # semantic_list = sorted(os.listdir(semantic_dir), key=lambda x: int(
    #     x.split("_")[-1].split(".")[0]))

    rgb_list = [os.path.join(rgb_dir, x) for x in rgb_list]
    depth_list = [os.path.join(depth_dir, x) for x in depth_list]
    pose_list = [os.path.join(pose_dir, x) for x in pose_list]
    # semantic_list = [os.path.join(semantic_dir, x) for x in semantic_list]


    map_save_dir = os.path.join(img_save_dir, "map")
    os.makedirs(map_save_dir, exist_ok=True)
    # color_top_down_save_path = os.path.join(map_save_dir, f"color_top_down_{mask_version}.npy")
    # # gt_save_path = os.path.join(map_save_dir, f"grid_{mask_version}_gt.npy")
    # grid_save_path = os.path.join(map_save_dir, f"grid_lseg_{mask_version}.npy")
    # weight_save_path = os.path.join(map_save_dir, f"weight_lseg_{mask_version}.npy")
    # obstacles_save_path = os.path.join(map_save_dir, "obstacles.npy")

    # obj2cls = load_obj2cls_dict(obj2cls_path)

    # initialize a grid with zero position at the center
    color_top_down_height = (camera_translation[2] + 1) * np.ones((gs, gs), dtype=np.float32)
    color_top_down = np.zeros((gs, gs, 3), dtype=np.uint8)
    # gt = np.zeros((gs, gs), dtype=np.int32)
    # grid = np.zeros((gs, gs, clip_feat_dim), dtype=np.float32)
    obstacles = np.ones((gs, gs), dtype=np.uint8)
    weight = np.zeros((gs, gs), dtype=float)

    # save_map(color_top_down_save_path, color_top_down)
    # # save_map(gt_save_path, gt)
    # save_map(grid_save_path, grid)
    # save_map(weight_save_path, weight)
    # save_map(obstacles_save_path, obstacles)

    tf_list = []
    # data_iter = zip(rgb_list, depth_list, semantic_list, pose_list)
    data_iter = zip(rgb_list, depth_list, pose_list)
    pbar = tqdm(total=len(rgb_list))
    # load all images and depths and poses
    # print('here')
    count = 0

    trans_cam = np.zeros((4,4))
    trans_cam[0, 2] = 1
    trans_cam[1, 0] = -1
    trans_cam[2, 1] = -1

    trans_cam[:3,3] = np.array(camera_translation)
    trans_cam[3,3] = 1

    pc_combined = []
    colors = []
    # feature_cloud = []
    
    for data_sample in data_iter:
        count += 1
        # if count % 15 != 0:
        #     continue

        feature_list = []
        pixel_coords = []
        # count += 1
        # if count % 3 != 0:
        #     continue
        # rgb_path, depth_path, semantic_path, pose_path = data_sample
        rgb_path, depth_path, pose_path = data_sample

        
        bgr = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        out = rgb.copy()

        # read pose
        # pos, rot = load_pose(pose_path)  # z backward, y upward, x to the right -> theirs
        pos, rot = load_pose(pose_path)  # x front, y left, z upward -> ours

        pose = np.eye(4)
        pose[:3, :3] = rot
        pose[:3, 3] = pos.reshape(-1)

        # pose = pose @ trans_cam

        tf_list.append(pose)
        if len(tf_list) == 1:
            init_tf_inv = np.linalg.inv(tf_list[0])

        tf = init_tf_inv @ pose @ trans_cam
        # tf = init_tf_inv @ pose   # uncomment, if pose = pose @ trans_cam

        # read depth
        depth = load_depth(depth_path)
        # print(f"depth shape = {depth.shape}")
        # break
        # read semantic
        # semantic = load_semantic(semantic_path)
        # semantic = cvt_obj_id_2_cls_id(semantic, obj2cls)

        # pix_feats = get_lseg_feat(model, rgb, labels, transform, crop_size, base_size, norm_mean, norm_std)
        # print(pix_feats.shape)
        # transform all points to the global frame
        pc, mask = depth2pc_realsense(depth)
        
        shuffle_mask = np.arange(pc.shape[1]) 
        np.random.shuffle(shuffle_mask)
        shuffle_mask = shuffle_mask[::depth_sample_rate]
        mask = mask[shuffle_mask]
        pc = pc[:, shuffle_mask]
        pc = pc[:, mask]

        color_local = get_color(rgb)
        color_local = color_local[:, shuffle_mask]
        color_local = color_local[:, mask]


        # o3d_pc = o3d.geometry.PointCloud()
        # o3d_pc.points = o3d.utility.Vector3dVector(pc.T)
        # o3d_pc.colors = o3d.utility.Vector3dVector((color_local / 255))
        # o3d_pc, _ = o3d_pc.remove_statistical_outlier(nb_neighbors = 15, std_ratio = 1.0)
        # pc = (np.asarray(o3d_pc.points)).T
        # color_local = (np.asarray(o3d_pc.colors)).T

        pc_global = transform_pc(pc, tf)

        # original_dims = (rgb.shape[1], rgb.shape[0])
        # rgb_cam_mat = get_real_cam_mat(original_dims, original_dims)
        # print(rgb_cam_mat)
        # feat_cam_mat = get_real_cam_mat(original_dims, (pix_feats.shape[3], pix_feats.shape[2]))
        # print(feat_cam_mat)

        if len(pc_combined) == 0:
            pc_combined = pc_global
            colors = color_local
        else:
            pc_combined = np.hstack((pc_combined, pc_global))
            # print(color_local.shape, colors.shape)
            colors = np.hstack((colors, color_local))
            
        # print(f"pc_local shape {pc.shape}")
        # project all point cloud onto the ground
        # print(np.max(pc[1]))
        # for i, (p, p_local) in enumerate(zip(pc_global.T, pc.T)):
            # x, y = pos2grid_id(gs, cs, p[0], p[2])   # if pose = pose @ trans_cam
            # x, y = pos2grid_id(gs, cs, p[0], p[1])   # changed p[2] -> p[1], i.e, z -> y

            # rgb_px, rgb_py, rgb_pz = project_point(rgb_cam_mat, p_local)
            # rgb_v = rgb[rgb_py, rgb_px, :]
            # colors.append(rgb_v)

            # ignore points projected to outside of the map and points that are 1.0 higher than the camera (could be from the ceiling)
            # if x >= obstacles.shape[0] or y >= obstacles.shape[1] or \
            #     x < 0 or y < 0 or p_local[1] < -1.0:
            #     continue

            # semantic_v = semantic[rgb_py, rgb_px]
            # if semantic_v == 40:
            #     semantic_v = -1
            
            # when the projected location is already assigned a color value before, overwrite if the current point has larger height
            # if p_local[1] < color_top_down_height[y, x]:
            #     color_top_down[y, x] = rgb_v
            #     color_top_down_height[y, x] = p_local[1]
                # gt[y, x] = semantic_v

            # average the visual embeddings if multiple points are projected to the same grid cell
            # px, py, pz = project_point(feat_cam_mat, p_local)
            # if not (px < 0 or py < 0 or px >= pix_feats.shape[3] or py >= pix_feats.shape[2]):
                # feat = pix_feats[0, :, py, px]
                # print(feat.shape)
                # feature_cloud.append(feat)
                # grid[y, x] = (grid[y, x] * weight[y, x] + feat) / (weight[y, x] + 1)
                # print(grid[y, x].shape)
                # weight[y, x] += 1
                
                # if count == 1:
                # pixel_coords.append([rgb_py, rgb_px])
                # feature_list.append(feat)
                # out = cv2.circle(out, (rgb_px, rgb_py), radius = 2, color = (255, 0, 0), thickness = 1)
            
            # build an obstacle map ignoring points on the floor (0 means occupied, 1 means free)
            # if p_local[1] > ground_to_camera_height:
            #     continue
            # obstacles[y, x] = 0

        # plt.imshow(out)
        # plt.show()
        pbar.update(1)
        np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
        np.save(os.path.join(map_save_dir, "colors.npy"), np.array(colors))
        # count += 1
        # break
        # if count == 75:
        #     break
        # print(len(pixel_coords))
        # print(feature_list.shape)

    # pc_combined = pc_combined @ trans_cam
    colors = np.array(colors)
    # pixel_coords = np.array(pixel_coords)
    # feature_list = np.array(feature_list)   
    # save_map(color_top_down_save_path, color_top_down)
    # save_map(gt_save_path, gt)
    # save_map(grid_save_path, grid)
    # save_map(weight_save_path, weight)
    # save_map(obstacles_save_path, obstacles)
    # np.save(os.path.join(map_save_dir, "pixel_coords.npy"), pixel_coords)
    # np.save(os.path.join(map_save_dir, "features_list.npy"), feature_list)
    # print(pixel_coords.shape)
    # print(feature_list.shape)
    np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
    np.save(os.path.join(map_save_dir, "colors.npy"), colors)

    torch.cuda.empty_cache()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
__file__:  /home2/laksh.nanwani/vlmaps-lab/examples/context.py
imported path: /home2/laksh.nanwani/vlmaps-lab


In [11]:
np.array([[1,2,3], [4,5,6]])[np.array([0,0]), np.array([1,0])]

array([2, 1])

In [12]:
root_dir = !pwd
print(root_dir)
root_dir = root_dir[0]
print(root_dir)
# !mkdir data
# %cd data
# !gdown 1wjuiVcO92Rqer5gLk-X7hINfe4PCMQmu
# !pip install tqdm
# !unzip -o 5LpN3gDmAk7_1.zip | tqdm --desc extracted --unit files --unit_scale --total `unzip -l 5LpN3gDmAk7_1.zip | tail -n 1 | xargs echo -n | cut -d' ' -f2` > /dev/null
import os
data_root = os.path.join(root_dir.split("home2")[0], "scratch/laksh.nanwani/vlmaps_data")

# checkpoint_dir = os.path.join(data_root, "checkpoints")
folder_name = "data_synced"
data_dir = os.path.join(data_root, folder_name)
# data_dir = os.path.join(data_root, "data_depth_issue")


# print(checkpoint_dir)
print(data_dir)

['/home2/laksh.nanwani/vlmaps-lab']
/home2/laksh.nanwani/vlmaps-lab
/scratch/laksh.nanwani/vlmaps_data/data_synced


In [13]:
# setup parameters
# @markdown meters per cell size
cs = 0.05 # @param {type: "number"}
# @markdown map resolution (gs x gs)
gs = 1000 # @param {type: "integer"}
# @markdown camera height (used for filtering out points on the floor)
# camera_height = 1.5 # @param {type: "number"}
# 0.51 - base_link -> camera_link
# camera_translation = [0.2, -0.222, 0.512]
camera_translation = [0.1, 0.018, 0.55]   # p3dx base_link to camera_link
# @markdown depth pixels subsample rate
depth_sample_rate = 100 # @param {type: "integer"}
# @markdown data where rgb, depth, pose are loaded and map are saved
data_dir = data_dir # @param {type: "string"}

In [14]:
pcd, colors = create_lseg_map_batch(data_dir, camera_translation=camera_translation, ground_to_camera_height = 0.5, cs=cs, gs=gs, depth_sample_rate=depth_sample_rate)

loading scene /scratch/laksh.nanwani/vlmaps_data/data_synced


  1%|▏         | 50/3403 [00:04<04:45, 11.76it/s]

KeyboardInterrupt: 

  1%|▏         | 51/3403 [00:20<04:44, 11.76it/s]

In [ ]:
pcd.shape
# colors.shape

(3, 399606)

In [ ]:
PC = o3d.geometry.PointCloud()
PC.points = o3d.utility.Vector3dVector(pcd.T)
o3d.visualization.draw_geometries([PC])

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
root_dir = !pwd
print(root_dir)
root_dir = root_dir[0]
print(root_dir)
# !mkdir data
# %cd data
# !gdown 1wjuiVcO92Rqer5gLk-X7hINfe4PCMQmu
# !pip install tqdm
# !unzip -o 5LpN3gDmAk7_1.zip | tqdm --desc extracted --unit files --unit_scale --total `unzip -l 5LpN3gDmAk7_1.zip | tail -n 1 | xargs echo -n | cut -d' ' -f2` > /dev/null
import os
data_root = os.path.join(root_dir.split("home2")[0], "scratch/laksh.nanwani/vlmaps_data")

# checkpoint_dir = os.path.join(data_root, "checkpoints")
data_dir = os.path.join(data_root, "5LpN3gDmAk7_1")

# print(checkpoint_dir)
print(data_dir)

['/home2/laksh.nanwani/vlmaps-lab']
/home2/laksh.nanwani/vlmaps-lab
/scratch/laksh.nanwani/vlmaps_data/5LpN3gDmAk7_1


In [ ]:
# setup parameters
# @markdown meters per cell size
cs = 0.05 # @param {type: "number"}
# @markdown map resolution (gs x gs)
gs = 1000 # @param {type: "integer"}
# @markdown camera height (used for filtering out points on the floor)
camera_height = 1.5 # @param {type: "number"}
# @markdown depth pixels subsample rate
depth_sample_rate = 50 # @param {type: "integer"}
# @markdown data where rgb, depth, pose are loaded and map are saved
data_dir = data_dir # @param {type: "string"}

In [ ]:
from utils.clip_mapping_utils import depth2pc as depth2pc_demo
from utils.clip_mapping_utils import load_pose as load_pose_demo

def load_depth_demo(depth_filepath):
    with open(depth_filepath, 'rb') as f:
        depth = np.load(f)
    return depth

def create_lseg_map_batch_demo(img_save_dir, camera_height, cs=0.05, gs=1000, depth_sample_rate=100):
    mask_version = 1 # 0, 1

    print(f"loading scene {img_save_dir}")
    rgb_dir = os.path.join(img_save_dir, "rgb")
    depth_dir = os.path.join(img_save_dir, "depth")
    pose_dir = os.path.join(img_save_dir, "pose")
    semantic_dir = os.path.join(img_save_dir, "semantic")
    # obj2cls_path = os.path.join(img_save_dir, "obj2cls_dict.txt")

    rgb_list = sorted(os.listdir(rgb_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    depth_list = sorted(os.listdir(depth_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    semantic_list = sorted(os.listdir(semantic_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))

    rgb_list = [os.path.join(rgb_dir, x) for x in rgb_list]
    depth_list = [os.path.join(depth_dir, x) for x in depth_list]
    pose_list = [os.path.join(pose_dir, x) for x in pose_list]
    semantic_list = [os.path.join(semantic_dir, x) for x in semantic_list]


    map_save_dir = os.path.join(img_save_dir, "map_demo")
    os.makedirs(map_save_dir, exist_ok=True)

    tf_list = []
    data_iter = zip(rgb_list, depth_list, semantic_list, pose_list)
    pbar = tqdm(total=len(rgb_list))
    # load all images and depths and poses
    print('here')

    pc_combined = []
    colors = []
    
    for data_sample in data_iter:
        rgb_path, depth_path, semantic_path, pose_path = data_sample
        
        bgr = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        # out = rgb.copy()

        # read pose
        pos, rot = load_pose_demo(pose_path)  # z backward, y upward, x to the right
        rot_ro_cam = np.eye(3)
        rot_ro_cam[1, 1] = -1
        rot_ro_cam[2, 2] = -1
        rot = rot @ rot_ro_cam
        pos[1] += camera_height


        pose = np.eye(4)
        pose[:3, :3] = rot
        pose[:3, 3] = pos.reshape(-1)

        tf_list.append(pose)
        if len(tf_list) == 1:
            init_tf_inv = np.linalg.inv(tf_list[0]) 

        tf = init_tf_inv @ pose

        # read depth
        depth = load_depth_demo(depth_path)
        # print(f"depth shape = {depth.shape}")
        
        # transform all points to the global frame
        pc, mask = depth2pc_demo(depth)
        print(pc.shape)

        # o3d.visualization.draw_geometries([pc])
        
        shuffle_mask = np.arange(pc.shape[1])
        np.random.shuffle(shuffle_mask)
        shuffle_mask = shuffle_mask[::depth_sample_rate]
        mask = mask[shuffle_mask]
        pc = pc[:, shuffle_mask]
        pc = pc[:, mask]
        print(pc.shape)

        color_local = get_color(rgb)
        color_local = color_local[:, shuffle_mask]
        color_local = color_local[:, mask]

        pc_global = transform_pc(pc, tf)

        if len(pc_combined) == 0:
            pc_combined = pc_global
            colors = color_local
        else:
            pc_combined = np.hstack((pc_combined, pc_global))
            # print(color_local.shape, colors.shape)
            colors = np.hstack((colors, color_local))

        pbar.update(1)
        np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
        np.save(os.path.join(map_save_dir, "colors.npy"), np.array(colors))
        # break

    np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
    np.save(os.path.join(map_save_dir, "colors.npy"), np.array(colors))

__file__:  /home2/laksh.nanwani/vlmaps-lab/examples/context.py
imported path: /home2/laksh.nanwani/vlmaps-lab


In [ ]:
create_lseg_map_batch_demo(data_dir, camera_height=camera_height, cs=cs, gs=gs, depth_sample_rate=depth_sample_rate)

loading scene /scratch/laksh.nanwani/vlmaps_data/5LpN3gDmAk7_1


here
(3, 777600)
(3, 0)
(3, 777600)


(3, 0)
(3, 777600)
(3, 0)
(3, 777600)


(3, 0)
(3, 777600)
(3, 0)
(3, 777600)


(3, 0)
(3, 777600)
(3, 0)
(3, 777600)


...

(3, 0)
(3, 777600)
(3, 0)
(3, 777600)


(3, 0)
(3, 777600)
(3, 0)
(3, 777600)


(3, 0)
(3, 777600)
(3, 0)
(3, 777600)


(3, 0)
(3, 777600)
(3, 0)


KeyboardInterrupt: 

In [2]:
root_dir = !pwd
print(root_dir)
root_dir = root_dir[0]
print(root_dir)
# !mkdir data
# %cd data
# !gdown 1wjuiVcO92Rqer5gLk-X7hINfe4PCMQmu
# !pip install tqdm
# !unzip -o 5LpN3gDmAk7_1.zip | tqdm --desc extracted --unit files --unit_scale --total `unzip -l 5LpN3gDmAk7_1.zip | tail -n 1 | xargs echo -n | cut -d' ' -f2` > /dev/null
import os
data_root = os.path.join(root_dir.split("home2")[0], "scratch/laksh.nanwani/vlmaps_data")

# checkpoint_dir = os.path.join(data_root, "checkpoints")
data_dir = os.path.join(data_root, "FloorPlan203")

# print(checkpoint_dir)
print(data_dir)

['/home2/laksh.nanwani/vlmaps-lab']
/home2/laksh.nanwani/vlmaps-lab
/scratch/laksh.nanwani/vlmaps_data/FloorPlan203


In [3]:
# setup parameters
# @markdown meters per cell size
cs = 0.05 # @param {type: "number"}
# @markdown map resolution (gs x gs)
gs = 1000 # @param {type: "integer"}
# @markdown camera height (used for filtering out points on the floor)
camera_height = 0.7 # @param {type: "number"}
# @markdown depth pixels subsample rate
depth_sample_rate = 50 # @param {type: "integer"}
# @markdown data where rgb, depth, pose are loaded and map are saved
data_dir = data_dir # @param {type: "string"}

In [21]:
from utils.clip_mapping_utils import depth2pc_ai2thor #, load_poseai2thor_personal
from utils.clip_mapping_utils import load_pose

def load_depth_demo(depth_filepath):
    with open(depth_filepath, 'rb') as f:
        depth = np.load(f)
    return depth

def create_lseg_map_batch_ai2thor(img_save_dir, camera_height, cs=0.05, gs=1000, depth_sample_rate=100):
    mask_version = 1 # 0, 1

    print(f"loading scene {img_save_dir}")
    rgb_dir = os.path.join(img_save_dir, "rgb")
    depth_dir = os.path.join(img_save_dir, "depth")
    pose_dir = os.path.join(img_save_dir, "pose")
    # semantic_dir = os.path.join(img_save_dir, "semantic")
    # obj2cls_path = os.path.join(img_save_dir, "obj2cls_dict.txt")

    rgb_list = sorted(os.listdir(rgb_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    depth_list = sorted(os.listdir(depth_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    pose_list = sorted(os.listdir(pose_dir), key=lambda x: int(
        x.split("_")[-1].split(".")[0]))
    # semantic_list = sorted(os.listdir(semantic_dir), key=lambda x: int(
    #     x.split("_")[-1].split(".")[0]))

    rgb_list = [os.path.join(rgb_dir, x) for x in rgb_list]
    depth_list = [os.path.join(depth_dir, x) for x in depth_list]
    pose_list = [os.path.join(pose_dir, x) for x in pose_list]
    # semantic_list = [os.path.join(semantic_dir, x) for x in semantic_list]


    map_save_dir = os.path.join(img_save_dir, "map_demo")
    os.makedirs(map_save_dir, exist_ok=True)

    tf_list = []
    data_iter = zip(rgb_list, depth_list, pose_list)
    pbar = tqdm(total=len(rgb_list))
    # load all images and depths and poses
    print('here')

    pc_combined = []
    colors = []
    
    count = 0
    for data_sample in data_iter:
        rgb_path, depth_path, pose_path = data_sample
        # print(data_sample)
        
        bgr = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        # out = rgb.copy()

        # read pose
        pos, rot = load_pose(pose_path)  # z backward, y upward, x to the right
        rot_ro_cam = np.eye(3)
        rot_ro_cam[1, 1] = -1
        rot_ro_cam[2, 2] = -1
        rot = rot @ rot_ro_cam
        pos[1] += camera_height

        # rot = rot.T @ rot_ro_cam
        # pos[2] = -pos[2]

        pose = np.eye(4)
        pose[:3, :3] = rot
        pose[:3, 3] = pos.reshape(-1)

        tf_list.append(pose)
        if len(tf_list) == 1:
            init_tf_inv = np.linalg.inv(tf_list[0]) 

        tf = init_tf_inv @ pose

        # read depth
        depth = load_depth_demo(depth_path)
        # print(f"depth shape = {depth.shape}")
        
        # transform all points to the global frame
        pc, mask = depth2pc_ai2thor(depth)
        # print(pc.shape)

        # o3d.visualization.draw_geometries([pc])
        
        shuffle_mask = np.arange(pc.shape[1])
        np.random.shuffle(shuffle_mask)
        shuffle_mask = shuffle_mask[::depth_sample_rate]
        mask = mask[shuffle_mask]
        pc = pc[:, shuffle_mask]
        pc = pc[:, mask]
        # print(pc.shape)

        color_local = get_color(rgb)
        color_local = color_local[:, shuffle_mask]
        color_local = color_local[:, mask]

        pc_global = transform_pc(pc, tf)

        if len(pc_combined) == 0:
            pc_combined = pc_global
            colors = color_local
        else:
            pc_combined = np.hstack((pc_combined, pc_global))
            # print(color_local.shape, colors.shape)
            colors = np.hstack((colors, color_local))

        pbar.update(1)
        np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
        np.save(os.path.join(map_save_dir, "colors.npy"), np.array(colors))
        # break

        if count == 10:
            break
        count += 1

    np.save(os.path.join(map_save_dir, "pointcloud.npy"), pc_combined)
    np.save(os.path.join(map_save_dir, "colors.npy"), np.array(colors))

In [22]:
create_lseg_map_batch_ai2thor(data_dir, camera_height=camera_height, cs=cs, gs=gs, depth_sample_rate=depth_sample_rate)

loading scene /scratch/laksh.nanwani/vlmaps_data/FloorPlan203


  0%|          | 2/986 [00:00<01:02, 15.64it/s]

here


  1%|          | 11/986 [00:00<01:22, 11.75it/s]
